# Week 8 Mini Project: Kmeans Clustering with Hadoop
Aaron Palumbo

## Overview

### Assign K

We need to tell the algorithm how many clusters we want.

### Initialize Clusters

* Pick k points randomly and assign those as cluster centers
* Broacast the cluster centers to all processors

### Map

For each row:
* calculate distance to each cluster center
* assign point to closest cluster
* key: cluster id
* value: coordinates of row

### Combine

Group keys (this will be according to cluster):
* key: cluster id
* value: (count of points in cluster, cluster center)

### Reduce

Group keys (this will be, again, according to cluster):
* Do a weighted average of cluster center to come up with new cluster centers

This is our new estimate of the cluster centers that we will use for the next iteration. All our processors will need access to this vector.

### Iterate until convergence

We continue these iterations until the centers move less than predifined convergence criteria, or unntil we reach a predefined number of iterations.

## Code

### Initialization

In [1]:
import numpy as np

# paramters
file_name = "birch1.txt"
k = 100 # number of clusters

**from mrjob can upload a list of files with --file: --file file_1.txt file_2.sqlite**

**then hopefully I can read this in where needed in my map job**

In [5]:
# these files aren't that big, but let's see if we
# can do this with a file we can't just read into memory

# get a line count
with open(file_name) as f:
    num_lines = sum(1 for line in f)

import random

offset = {random.randrange(num_lines) for i in xrange(k)}
assert(len(offset) == k) # stop if  we have a collision

clusters = []
with open(file_name) as f:
    for i, line in enumerate(f):
        if i in offset:
            clusters.append(line.split())

lines = []
with open(file_name) as f:
    for i, line in enumerate(f):
        if i in offset:
            lines.append(line)

clusters = np.array(clusters).astype('float')      


SyntaxError: invalid syntax (<ipython-input-5-4d3e77faaf31>, line 25)

In [3]:
%%writefile kmeans.py
from mrjob.job import MRJob
import numpy as np

class KMeans(MRJob):
    
    
    def mapper(self, _, line):
        x = np.array(line.split()).astype('float')
        if x in clusters:
            yield "chars", len(line)
            yield "words", len(line.split())
            yield "lines", 1
        
    def reducer(self, key, values):
        yield key, sum(values)
        
if __name__ == "__main__":
    KMeans.run()

Overwriting kmeans.py


In [4]:
# run KMeans
from kmeans import KMeans
mr_job = KMeans(args=["--output-dir", "mrjob_output", "birch3.txt"])
with mr_job.make_runner() as runner:
    runner.run()

NameError: global name 'clusters' is not defined

In [ ]:
mrjob_output = !cat mrjob_output/part-00000
for line in mrjob_output:
    print line